In [1]:
import os
os.chdir(os.path.dirname(os.path.abspath("__file__")))


In [2]:
import sys
print(sys.executable)
print(sys.path)

c:\Git_Tutorials\IntentClassification\.venv\Scripts\python.exe
['C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Git_Tutorials\\IntentClassification\\.venv', '', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\win32', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\Pythonwin']


In [3]:
import pandas as pd
import numpy as np
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
from pathlib import Path
import requests
from zipfile import ZipFile


In [4]:
# code to convert dataframe records to json
def csv_to_json_converter(csv_file_path):
    try:
        local_df = pd.read_csv(csv_file_path)
        local_df.reset_index(drop=True, inplace=True)
        records = local_df.to_dict(orient='records')
        return records
    except Exception as e:
        raise e

In [5]:
#code to insert the jsons into mongodb

def insert_data_mongodb(json_data, database, collection):
    try:
        #connect to MongoDB
        load_dotenv()
        client = MongoClient(os.getenv('MONGO_DB_URL'), server_api=ServerApi('1'))
        client.admin.command('ping')
        print("Connected to MongoDB")
        db = client[database]
        colln = db[collection]
        #insert the json data
        colln.insert_many(json_data)
        return(f"Inserted {len(json_data)} records into {collection} collection in {database} database.")
    except Exception as e:
        raise e

In [6]:
def export_collection_as_csv(csv_file_path, database, collection):
    try:
        load_dotenv()
        client = MongoClient(os.getenv('MONGO_DB_URL'), server_api=ServerApi('1'))
        client.admin.command('ping')
        print("Connected to MongoDB")
        db = client[database]
        colln=db[collection]
        
        df = pd.DataFrame(list(colln.find()))
        print("export successful")
        if "_id" in df.columns:
            df.drop(columns=['_id'], inplace =True, axis=1)
        if "Unnamed: 0" in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace =True, axis=1)
        print("csv_file saved succesfully")
        df.to_csv(csv_file_path, index=False, header = True)

    except Exception as e:
        print(e)
        raise e

In [7]:
from src.IntentClassification.utils.common import read_yaml
from src.IntentClassification import constants as const
import re

sentence = "perform partial redemption using service dptdrecordarredemption"

sentence = re.sub(r'\bdp\w+','web service',sentence)
print(sentence)



perform partial redemption using service web service


In [8]:
def download_glove_embeddings(url, glove_folder, glove_zip_file):
    if not os.path.exists(glove_folder):
        os.makedirs(glove_folder, exist_ok=True)
        response = requests.get(url, stream=True)
        with open(glove_zip_file, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded glove embeddings from {url}")
        with ZipFile(glove_zip_file, 'r') as zip:
            zip.extractall(glove_folder)
        print(f"Unzipped glove embeddings to {glove_folder}")


In [9]:
url = "http://nlp.stanford.edu/data/glove.6B.zip"
glove_folder = "artifacts/glove"
glove_zip_file = "artifacts/glove/glove.6B.zip"

download_glove_embeddings(url, glove_folder, glove_zip_file)

In [10]:
def load_glove_embeddings(file_path="glove_data/glove.6B.100d.txt"):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float32)
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings


In [11]:
glove_path = "artifacts/glove/glove.6B.100d.txt"

glove_embeddings = load_glove_embeddings(glove_path)

Loaded 400000 word vectors.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
df = pd.read_csv('artifacts/data_preprocessing/preprocessed.csv')
corpus = df['preprocessed'].astype(str).tolist()

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
idf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(),tfidf_vectorizer.idf_))


In [13]:
def sentence_to_weighted_glove(sentence, glove_embeddings, idf_scores):
    words = sentence.split()
    vector = np.zeros(100)  # Assuming 100-dimensional GloVe embeddings
    total_weight = 0
    to_be_vectors = []

    for word in words:
        print(word)
        if word in glove_embeddings and word in idf_scores:
            weight = (idf_scores[word])
            vector += glove_embeddings[word]*weight
            total_weight += weight
        else:
            to_be_vectors.append(word)
            print(len(to_be_vectors))
            print(to_be_vectors)
    return vector / total_weight if total_weight != 0 else vector


In [14]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess


In [15]:
corpus = ['create a checking_account', 'place account_freeze restriction']

In [16]:
from nltk.tokenize import word_tokenize

def tokenize_corpus(corpus):
    return [word_tokenize(text) for text in corpus]

In [17]:
word_2_vec_path = "artifacts/word2vec.model"

def save_word2vec_model(corpus, model_path):
    model = Word2Vec(sentences=tokenize_corpus(corpus), vector_size=100, window=5, min_count=1, workers=4)
    model.save(model_path)

def load_word2vec_model(model_path):
    return Word2Vec.load(model_path)

In [18]:
save_word2vec_model(corpus, word_2_vec_path)

In [19]:
def sentence_to_weighted_vectors(sentence, glove_embeddings, idf_scores, model_path):
    words = word_tokenize(sentence)
    vector = np.zeros(100)  # Assuming 100-dimensional GloVe embeddings
    total_weight = 0

    word_2_vec = load_word2vec_model(model_path)

    for word in words:
        
        if word in glove_embeddings and word in idf_scores:
            weight = (idf_scores[word])
            vector += glove_embeddings[word]*weight
            total_weight += weight
        else:
            print(word)
            vector += word_2_vec.wv[word]
    return vector / total_weight if total_weight != 0 else vector

In [20]:
vec = {}
for sentence in corpus:
    vector = sentence_to_weighted_vectors(sentence, glove_embeddings, idf_scores, word_2_vec_path)
    vec[sentence] = vector

a
checking_account
account_freeze


In [61]:
train = pd.read_csv('artifacts/data_transformation/train.csv')


In [ ]:
train['vec'] = (train['vectors'].strip('[]')).replace("\n","")

In [65]:
train['vec'][0]

'-2.42688832e-01  1.22891995e-01  2.55557793e-01  4.63965025e-02 -6.75479614e-02  6.56613478e-02 -1.46482367e-01  2.08959666e-01  1.02084353e-01  3.38134380e-01  3.31140750e-01 -2.66453737e-02  3.41670768e-01 -2.15196002e-01  3.68941003e-02  4.98872425e-02  3.97148334e-01  2.18025527e-02 -8.70026599e-02 -1.86981713e-01  3.84414093e-02 -7.77777706e-02  3.24947738e-02  1.07738286e-01  5.97823742e-02 -5.63910908e-01 -9.65115131e-03 -2.59708427e-01 -1.28052417e-01 -1.09125040e-01  1.23917143e-01  1.82832194e-01 -1.21953672e-01 -2.75788116e-02  2.54936654e-01  1.11429589e-01  8.03972836e-02 -2.28821319e-02 -3.43967846e-01 -1.73367095e-01  1.36651585e-02 -2.78128162e-01  5.83898087e-02 -1.04438388e-01  4.14190777e-02  1.75811427e-02 -3.05580303e-01 -1.03667764e-01 -8.95829618e-02 -1.99510277e-01 -2.32906031e-02  1.33581449e-01 -7.84943385e-02  6.09820835e-01 -1.03401952e-01 -1.17174166e+00  1.04559154e-01 -4.00787765e-01  1.06850625e+00  1.82473974e-01 -2.46989819e-01  4.94643192e-01  8.4923

In [57]:
import ast

In [70]:
from src.IntentClassification import logger

In [ ]:
def convert_to_ndarray( X):
    try:
        logger.info("Inside convert_to_ndarray")
        X['vectors'] = X['vectors'].apply(lambda x: x.strip('[]').replace("\n",""))
        X['vectors'] = X['vectors'].apply(lambda x: np.fromstring(x, sep= ' '))
        print(type(X['vectors'].iloc[0]))
        
        return X['vectors']
    except Exception as e:
        logger.error(f"error occured: {e}")
        raise e

In [74]:
X = convert_to_ndarray(train)

[ 2025-09-01 20:05:35,445 ] [ INFO ][ 3 ] [ src.IntentClassification ] - Inside convert_to_ndarray


<class 'numpy.ndarray'>


In [77]:
print(np.array(X))

[array([-2.42688832e-01,  1.22891995e-01,  2.55557793e-01,  4.63965025e-02,
        -6.75479614e-02,  6.56613478e-02, -1.46482367e-01,  2.08959666e-01,
         1.02084353e-01,  3.38134380e-01,  3.31140750e-01, -2.66453737e-02,
         3.41670768e-01, -2.15196002e-01,  3.68941003e-02,  4.98872425e-02,
         3.97148334e-01,  2.18025527e-02, -8.70026599e-02, -1.86981713e-01,
         3.84414093e-02, -7.77777706e-02,  3.24947738e-02,  1.07738286e-01,
         5.97823742e-02, -5.63910908e-01, -9.65115131e-03, -2.59708427e-01,
        -1.28052417e-01, -1.09125040e-01,  1.23917143e-01,  1.82832194e-01,
        -1.21953672e-01, -2.75788116e-02,  2.54936654e-01,  1.11429589e-01,
         8.03972836e-02, -2.28821319e-02, -3.43967846e-01, -1.73367095e-01,
         1.36651585e-02, -2.78128162e-01,  5.83898087e-02, -1.04438388e-01,
         4.14190777e-02,  1.75811427e-02, -3.05580303e-01, -1.03667764e-01,
        -8.95829618e-02, -1.99510277e-01, -2.32906031e-02,  1.33581449e-01,
        -7.8

In [28]:
import ast


In [42]:
train['vectors'] = train['vectors'].apply(lambda x: np.fromstring(x.strip('[]').replace('\n', ' '), sep=' '))
#train['vectors'] = train['vectors'].apply(lambda x: np.array(ast.literal_eval(x)))

In [43]:
train['vectors'] = np.vstack(train['vectors'].values)

In [44]:
train['vectors'][0]

-0.242688832